<a href="https://colab.research.google.com/github/VKSi/2020_12_GB_RecommendationSystems/blob/main/capstone_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install implicit

     |████████████████████████████████| 1.1MB 10.3MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419433 sha256=813592279b4a4af59f302bcc7b80c284ad6783e1415ed442749949642254ef2d
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
PATH = 'gdrive/My Drive/AI/GB/RecSys/capstone/'
OBJECTS_PATH = PATH + 'objects/'
DATA_PATH = PATH + 'raw_data/'

In [ ]:
def save_obj(obj, name: str):
    path = OBJECTS_PATH + name + '.pkl'
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    return path

def load_obj(name: str):
    with open(OBJECTS_PATH + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
# Функции из модуля metrics
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    

    bought_list = bought_list  # Тут нет [:k] !!
    
    if k < len(recommended_list):
        recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    # your_code
    # Лучше считать через скалярное произведение, а не цикл

    return precision


def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    recall = flags.sum() / len(bought_list)

    return recall


def recall_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    if k < len(recommended_list):
        recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)

    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    # your_code

    return recall

In [ ]:
# Функции из модуля utils

def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

    top_popular = popularity[popularity['share_unique_users'] > 0.2].item_id.tolist()
    # data = data[~data['item_id'].isin(top_popular)]                                                 # !!! НЕ ДЕЛАЕМ !!! 

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.02].item_id.tolist()
    # data = data[~data['item_id'].isin(top_notpopular)]                                                 # !!! НЕ ДЕЛАЕМ !!!

    # Уберем товары, которые не продавались за последние 12 месяцев

    # Уберем не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features.\
                                        groupby('department')['item_id'].nunique().\
                                        sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        # data = data[~data['item_id'].isin(items_in_rare_departments)]                                                 # !!! НЕ ДЕЛАЕМ !!!


    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    # data = data[data['price'] > 2]                                                 # !!! НЕ ДЕЛАЕМ !!!

    # Уберем слишком дорогие товарыs
    data = data[data['price'] < 50]

    # Возбмем топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()
    
    # Заведем фиктивный item_id (если юзер не покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999
    
    # ...

    return data


def postfilter_items(user_id, recommednations):
    pass

In [ ]:
# Функции из модуля recommenders

import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads,
                                        use_gpu=False)  # Добавлено, чтобы избежать ошибки с CUDA
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def _update_dict(self, user_id):
        """Если появился новыю user / item, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():

            max_id = max(list(self.userid_to_id.values()))
            max_id += 1

            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[1][0]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            recommendations.extend(self.overall_top_purchases[:N])
            recommendations = recommendations[:N]

        return recommendations

    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        res = [self.id_to_itemid[rec[0]] for rec in model.recommend(userid=self.userid_to_id[user],
                                        user_items=csr_matrix(self.user_item_matrix).tocsr(),
                                        N=N,
                                        filter_already_liked_items=False,
                                        filter_items=[self.itemid_to_id[999999]],
                                        recalculate_user=True)]

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.model, N=N)

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.own_recommender, N=N)

    def get_similar_items_recommendations(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N)

        res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendations(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        res = []

        # Находим топ-N похожих пользователей
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)
        similar_users = [rec[0] for rec in similar_users]
        similar_users = similar_users[1:]   # удалим юзера из запроса

        for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

In [ ]:
data = pd.read_csv(DATA_PATH + 'retail_train.csv')
data_lb = pd.read_csv(DATA_PATH + 'retail_test1.csv')

item_features = pd.read_csv(DATA_PATH + 'product.csv')
user_features = pd.read_csv(DATA_PATH + 'hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


In [ ]:
# !!! Разбивка сделана не так как в методичке !!! В части data_val_lvl_1 взят весь остаток, data_train_lvl_2 считается отдельно

val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]

data_train_lvl_2 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


Посмотрим на тестовые (лидерборд) данные

In [ ]:
data_lb.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0


In [ ]:
users_t = set(data['user_id'].unique())
userd_lb = set(data_lb['user_id'].unique())
new_users = userd_lb - users_t
print(f'users_t: {len(users_t)}')
print(f'userd_lb: {len(userd_lb)}')
print(f'new_users: {len(new_users)}')
print(new_users)

users_t: 2499
userd_lb: 1885
new_users: 1
{2325}


In [ ]:
train_users = set(data_train_lvl_1['user_id'].unique())
userd_lb = set(data_lb['user_id'].unique())
new_users = userd_lb - train_users
print(f'train_users: {len(train_users)}')
print(f'userd_lb: {len(userd_lb)}')
print(f'new_users: {len(new_users)}')
print(new_users)

train_users: 2498
userd_lb: 1885
new_users: 1
{2325}


То есть в сете для оценки у нас только один новый юзер 2325

In [ ]:
data.week_no.max()

95

In [ ]:
data_lb.week_no.max()

98

Данные на лидерборде содержат 3 недели покупок

Сделаем модель минимальным предварительным фильтром (см код фильтра в классе)

In [ ]:
train_users = set(data_train_lvl_1['user_id'].unique())
val_users = set(data_val_lvl_1['user_id'].unique())
new_users = val_users - train_users
print(f'train_users: {len(train_users)}')
print(f'val_users: {len(val_users)}')
print(f'new_users: {len(new_users)}')

train_users: 2498
val_users: 2280
new_users: 1


In [ ]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=30000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Decreased # items from 83685 to 30001


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
train_users = set(data_train_lvl_1['user_id'].unique())
val_users = set(data_val_lvl_1['user_id'].unique())
new_users = val_users - train_users
print(f'train_users: {len(train_users)}')
print(f'val_users: {len(val_users)}')
print(f'new_users: {len(new_users)}')

train_users: 2498
val_users: 2280
new_users: 1


In [ ]:
# recommender = MainRecommender(data_train_lvl_1)

In [ ]:
# save_obj(recommender, 'recommender_min_prefilter_30000')

'gdrive/My Drive/AI/GB/RecSys/capstone/objects/recommender_min_prefilter_30000.pkl'

In [ ]:
recommender = load_obj('recommender_min_prefilter_30000')

In [ ]:
recommender.get_als_recommendations(2375, N=5)

[981760, 1082185, 923746, 1098066, 1037863]

In [ ]:
recommender.get_own_recommendations(2375, N=5)

[948640, 5565086, 1069256, 824738, 1335640]

In [ ]:
recommender.get_similar_items_recommendations(2375, N=5)

[820352, 889731, 1046545, 1055646, 9527160]

In [ ]:
recommender.get_similar_users_recommendations(2375, N=5)

[1093490, 909618, 985605, 5585068, 979674]

In [ ]:
data_train_lb = prefilter_items(data.copy(deep=True), item_features=item_features, take_n_popular=30000)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
recommender_lb = MainRecommender(data_train_lb)

In [ ]:
save_obj(recommender_lb, 'recommender_min_prefilter_30000_lb')

'gdrive/My Drive/AI/GB/RecSys/capstone/objects/recommender_min_prefilter_30000_lb.pkl'

In [ ]:
recommender_lb = load_obj('recommender_min_prefilter_30000_lb')

# Отбор первого уровня


In [ ]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [ ]:
actual_users = set(result_lvl_1['user_id'].unique())
predicted_users = set(data_train_lvl_1['user_id'].unique())
new_users_1 = actual_users - predicted_users
print(f'actual_users: {len(actual_users)}')
print(f'predicted_users: {len(predicted_users)}')
print(f'new_users_1: {len(new_users_1)}')

actual_users: 2280
predicted_users: 2498
new_users_1: 1


In [ ]:
result_lvl_1_lb = data_lb.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1_lb.columns=['user_id', 'actual']
result_lvl_1_lb.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [ ]:
k = 50

Напишем полезные функции для автоматизации дальнейшей работы

In [ ]:
def get_bought_list(actual_data, user_id):
  return actual_data.loc[actual_data['user_id'] == user_id, 'actual'].values[0]

def get_recommended_list(func_name: str, user_id, N):
  return eval(f'recommender.get_{func_name}_recommendations({user_id}, {N})')

def total_recall_at_k(actual_data, k, N, func_name: str):
  rc = []
  for uid in set(recommender.id_to_userid.values()):
    if uid not in actual_data['user_id'].values:
      continue
    rl = get_recommended_list(func_name=func_name, user_id=uid, N=N)
    bl = get_bought_list(actual_data=actual_data, user_id=uid)
    rc.append(recall_at_k(rl, bl, k))
  return np.mean(rc), np.std(rc)

In [ ]:
to_count = result_lvl_1['user_id'].isin(predicted_users)

In [ ]:
def prepare_lvl_1_candidates_table(recommender, actual_data, N, candidates_table_name):
  candidates = pd.DataFrame(actual_data['user_id'].unique())
  candidates.columns = ['user_id']
  candidates = candidates.sort_values(by='user_id', ascending=True)
  users = candidates.user_id.values
  
  top_rec = recommender.overall_top_purchases
  candidates[f'top_{N}'] = candidates['user_id'].apply(lambda x: recommender.overall_top_purchases[:N])

  for func_name in ['als', 'own', 'similar_items']:
    i = 0
    for uid in users:
      i += 1
      try:
        candidates.loc[candidates.user_id == uid, f'{func_name}_{N}'] = \
              candidates.loc[candidates.user_id == uid, 'user_id'].apply(lambda x: get_recommended_list(func_name=func_name, user_id=x, N=N))
      except:
        candidates.loc[candidates.user_id == uid, f'{func_name}_{N}'] = np.nan
      if i % 300 == 0:
        print(i)
    # candidates.to_csv(OBJECTS_PATH + f'candidates_{candidates_table_name}.csv')
    save_obj(candidates, f'candidates_{candidates_table_name}_obj')
    print(f'{func_name} сохранено')
  
  return candidates


In [ ]:
candidates_df = prepare_lvl_1_candidates_table(recommender=recommender, actual_data=data_val_lvl_1, N=100, candidates_table_name='model_100_step_0')

300
600
900
1200
1500
1800
2100
als сохранено
300
600
900
1200
1500
1800
2100
own сохранено
300
600
900
1200
1500
1800
2100
similar_items сохранено


In [ ]:
candidates_df.isna().sum()

user_id              0
top_100              0
als_100              1
own_100              1
similar_items_100    0
dtype: int64

In [ ]:
candidates_df[candidates_df['als_100'].isna()]

,user_id,top_100,als_100,own_100,similar_items_100
1903,1984,"[1082185, 6534178, 1029743, 995242, 1106523, 9...",NaN,NaN,"[1082185, 6534178, 1029743, 995242, 1106523, 9..."


In [ ]:
candidates_df.head()

,user_id,top_100,als_100,own_100,similar_items_100
877,1,"[1082185, 6534178, 1029743, 995242, 1106523, 9...","[1082185, 1006546, 1075074, 5577022, 9858900, ...","[1049998, 852662, 896666, 1087895, 900875, 938...","[941313, 840361, 1077555, 844179, 1006414, 557..."
1769,2,"[1082185, 6534178, 1029743, 995242, 1106523, 9...","[1082185, 1098066, 951590, 981760, 1127831, 82...","[242044, 9651582, 73207, 1107760, 7463018, 238...","[8090509, 979098, 5592931, 7025275, 985999, 93..."
2178,3,"[1082185, 6534178, 1029743, 995242, 1106523, 9...","[826249, 1098066, 883404, 5568378, 1044078, 98...","[7443137, 835476, 6632283, 1011504, 10285454, ...","[1053690, 1092026, 960318, 892008, 925178, 979..."
1629,4,"[1082185, 6534178, 1029743, 995242, 1106523, 9...","[951590, 883932, 826249, 8090521, 1137346, 109...","[6391541, 1013868, 943030, 979674, 891423, 104...","[835530, 6391551, 8019276, 7138817, 8069350, 1..."
917,6,"[1082185, 6534178, 1029743, 995242, 1106523, 9...","[1082185, 878996, 1037863, 863632, 981760, 102...","[1405491, 876626, 998519, 849618, 919815, 1126...","[840361, 874149, 882138, 845208, 948650, 70252..."


In [ ]:
candidates_df_lb = prepare_lvl_1_candidates_table(recommender=recommender_lb, actual_data=data_lb, N=100, candidates_table_name='model_100_step_0_lb')

300
600
900
1200
1500
1800
als сохранено
300
600
900
1200
1500
1800
own сохранено
300
600
900
1200
1500
1800
similar_items сохранено


In [ ]:
def get_ind(itms, x, N):
  if x in itms:
    return itms.index(x)
  else:
    return N*1.5


def mix_values(candidates, user_id, N_used, N_return):
  cr = candidates[candidates.user_id == user_id]
  items = set()
  cols = cr.columns[1:]
  for col in cr.columns[1:]:
    if type(cr[col].values[0]) == type(np.nan):
      continue
    else:
      val = cr[col].values[0]
      items = items.union(set(val))

  mix_table = pd.DataFrame(columns = cr.columns[1:])
  mix_table['item_id'] = list(items)

  for col in cr.columns[1:]:
    if type(cr[col].values[0]) == type(np.nan):
      val = []
    else:
      val = cr[col].values[0]
    
    mix_table[col] = mix_table['item_id'].apply(lambda x: get_ind(val, x, N_used))

  a =  mix_table.drop(columns='item_id').sum(axis='columns')
  mix_table['mix'] = a

  mix_table = mix_table[['item_id', 'mix']].sort_values(by='mix', ascending=True).head(N_return)

  return mix_table.item_id.values


def get_mix(candidates, N_used, N_return):
  cand = candidates.sort_values(by='user_id', ascending=True).copy(deep=True)
  users = cand.user_id.values
  cand['mix'] = 0

  i = 0
  for uid in users:
    i += 1
    cand.loc[cand.user_id == uid, 'mix'] = \
            cand.loc[cand.user_id == uid, 'user_id'].apply(lambda x: mix_values(cand.drop(columns=['mix']), x, N_used, N_return))
    if i % 100 == 0:
      print(i)

  return cand 

In [ ]:
candidates_df_mix = get_mix(candidates_df, 100, 100)
save_obj(candidates_df_mix, f'candidates_model_100_mix_obj')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


'gdrive/My Drive/AI/GB/RecSys/capstone/objects/candidates_model_100_mix_obj.pkl'

In [ ]:
predicted_users_mix = set(candidates_df_mix['user_id'].unique())
len(predicted_users_mix)

2280

In [ ]:
actual_users - predicted_users_mix

set()

In [ ]:
len(candidates_df_mix.loc[1984, 'mix'])

100

In [ ]:
len(set(candidates_df_mix.loc[1984, 'mix']))

100

Видно, что "холодные" юзеры тоже заполнились

In [ ]:
candidates_df_mix_lb = get_mix(candidates_df_lb, 100, 100)
save_obj(candidates_df_mix_lb, f'candidates_model_100_mix_lb_obj')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800


'gdrive/My Drive/AI/GB/RecSys/capstone/objects/candidates_model_100_mix_lb_obj.pkl'

In [ ]:
# Определим функцию для расчета precision_at_k по готовому предсказанию
def total_precision_at_k_pre(actual_data, k, recommendations_table):
  pr = []
  for uid in actual_data['user_id'].values:
    if uid == 1984 or uid == 2325:
      continue
    rl = recommendations_table.loc[recommendations_table.user_id == uid, 'candidates'].values[0]
    bl = get_bought_list(actual_data, user_id=uid)
    pr.append(precision_at_k(recommended_list=rl, bought_list=bl, k=k))
  return np.mean(pr), np.std(pr)

# Определим функцию для расчета recall_at_k по готовому предсказанию
def total_recall_at_k_pre(actual_data, k, recommendations_table):
  pr = []
  for uid in actual_data['user_id'].values:
    if uid == 1984 or uid == 2325:
      continue
    rl = recommendations_table.loc[recommendations_table.user_id == uid, 'candidates'].values[0]
    bl = get_bought_list(actual_data, user_id=uid)
    pr.append(recall_at_k(recommended_list=rl, bought_list=bl, k=k))
  return np.mean(pr), np.std(pr)

In [ ]:
for col in candidates_df_mix.columns[1:]:
  data = candidates_df_mix[['user_id', col]].rename(columns={col:'candidates'})
  mean, std = total_recall_at_k_pre(actual_data = result_lvl_1, k=100, recommendations_table=data)
  print(col, mean, ' +- ', std)

top_100 0.11473904383994822  +-  0.09558397890830696
als_100 0.1001994635446865  +-  0.08452658275099939
own_100 0.07170587667316233  +-  0.08770463540904594
similar_items_100 0.06357626451448317  +-  0.06908941582164613
mix 0.1361729681160206  +-  0.10409861318807474


Recall маловат. Разброс большой. MIX показал лучший результат

In [ ]:
for col in candidates_df_mix.columns[1:]:
  data = candidates_df_mix[['user_id', col]].rename(columns={col:'candidates'})
  mean, std = total_precision_at_k_pre(actual_data = result_lvl_1, k=5, recommendations_table=data)
  print(col, mean, ' +- ', std)

top_100 0.2840719613865731  +-  0.24005613300552042
als_100 0.25730583589293554  +-  0.2551065628654374
own_100 0.1601579640193067  +-  0.2074335359680525
similar_items_100 0.10978499341816587  +-  0.1619481477680439
mix 0.26239578762615184  +-  0.2521755441074338


Precision на модели первого уровня уже дает 28% для top и 26% для mix. Однако сильно смущает вариация.

Сразу проверим liderboard. Вдруг модели первого уровня окажется достаточно:

In [ ]:
for col in candidates_df_mix_lb.columns[1:]:
  data = candidates_df_mix_lb[['user_id', col]].rename(columns={col:'candidates'})
  mean, std = total_recall_at_k_pre(actual_data = result_lvl_1_lb, k=100, recommendations_table=data)
  print(col, mean, ' +- ', std)

top_100 0.14649554015063873  +-  0.13716848490015132
als_100 0.12162860613981015  +-  0.11877085380467681
own_100 0.073824400096259  +-  0.10553557669729863
similar_items_100 0.07110102395345949  +-  0.08623168479038966
mix 0.16732702214559877  +-  0.14484699328641465


In [ ]:
for col in candidates_df_mix_lb.columns[1:]:
  data = candidates_df_mix_lb[['user_id', col]].rename(columns={col:'candidates'})
  mean, std = total_precision_at_k_pre(actual_data = result_lvl_1_lb, k=5, recommendations_table=data)
  print(col, mean, ' +- ', std)

top_100 0.1937367303609342  +-  0.19544452764272477
als_100 0.16422505307855628  +-  0.19607178450812063
own_100 0.090552016985138  +-  0.1513418851151237
similar_items_100 0.0564755838641189  +-  0.11636524120797183
mix 0.15690021231422507  +-  0.18877791302431224


Маловато будет )))

In [ ]:
users_lvl_2 = candidates_df_mix[['user_id', 'mix']].rename(columns={'mix':'candidates'})
save_obj(users_lvl_2, 'users_lvl_2_obj')

'gdrive/My Drive/AI/GB/RecSys/capstone/objects/users_lvl_2_obj.pkl'

In [ ]:
users_lvl_2_lb = candidates_df_mix_lb[['user_id', 'mix']].rename(columns={'mix':'candidates'})
save_obj(users_lvl_2_lb, 'users_lvl_2_lb_obj')